In [3]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import * 
from pyspark.sql.functions import regexp_replace

In [4]:
spark = SparkSession.builder \
.master("local[2]") \
.appName("Clean_Data") \
.getOrCreate()

2023-02-22 19:30:40,985 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-02-22 19:30:45,055 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
! wget -P /home/train/datasets/ \
https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv

--2023-02-22 14:43:23--  https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/master/dirty_store_transactions.csv [following]
--2023-02-22 14:43:24--  https://raw.githubusercontent.com/erkansirin78/datasets/master/dirty_store_transactions.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2609524 (2.5M) [text/plain]
Saving to: ‘/home/train/datasets/dirty_store_transactions.csv’

100%[======================================>] 2,609,524   3.15MB/s   in 0.8s   

2023-02-22 14:43:25 (3.15 MB/s) - ‘/home

# Read Data

In [5]:
df = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/dirty_store_transactions.csv")

In [7]:
df.limit(5).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,New York(,Electronics,12254943,$31,$20.77,$1.86,$29.14,2019-11-26
1,YR7220,New York+,Furniture,72619323C,$15,$9.75,$1.5,$13.5,2019-11-26
2,YR7220,New York,Electronics,34161682B,$88,$62.48,$4.4,$83.6,2019-11-26
3,YR7220,New York!,Kitchen,79411621,$91,$58.24,$3.64,$87.36,2019-11-26
4,YR7220,New York,Fashion,39520263T,$85,$51,$2.55,$82.45,2019-11-26


In [8]:
df.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|     New York(|     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|     New York+|       Furniture| 72619323C|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|     New York |     Electronics| 34161682B|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|     New York!|         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T|$85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|$37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D|$80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|     New York$|         Kitchen| 52503356^|$71| $42.6|   $5.68|$65.32|

In [9]:
from pyspark.sql.types import *

In [10]:
programmatical_schema = StructType([
        StructField("STORE_ID",StringType(),True),
        StructField("STORE_LOCATION",StringType(),True),
        StructField("PRODUCT_CATEGORY",StringType(),True),
        StructField("PRODUCT_ID",StringType(),True),
        StructField("MRP",StringType(),True),
        StructField("CP",StringType(),True),
        StructField("DISCOUNT",StringType(),True),
        StructField("SP",StringType(),True),
        StructField("Date",DateType(),True)
    ])

In [11]:
df2 = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.schema(programmatical_schema) \
.load("file:///home/train/datasets/dirty_store_transactions.csv")

In [12]:
df2.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: string (nullable = true)
 |-- CP: string (nullable = true)
 |-- DISCOUNT: string (nullable = true)
 |-- SP: string (nullable = true)
 |-- Date: date (nullable = true)



In [13]:
df2.show(n=4, truncate=False)

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|CP    |DISCOUNT|SP    |Date      |
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|YR7220  |New York(     |Electronics     |12254943  |$31|$20.77|$1.86   |$29.14|2019-11-26|
|YR7220  |New York+     |Furniture       |72619323C |$15|$9.75 |$1.5    |$13.5 |2019-11-26|
|YR7220  |New York      |Electronics     |34161682B |$88|$62.48|$4.4    |$83.6 |2019-11-26|
|YR7220  |New York!     |Kitchen         |79411621  |$91|$58.24|$3.64   |$87.36|2019-11-26|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
only showing top 4 rows



In [64]:
df2.count()

37853

In [14]:
symbol_pattern = r"[^\w\s]"

In [15]:
df3 = df2.withColumn("STORE_LOCATION", regexp_replace("STORE_LOCATION", symbol_pattern, ""))

In [18]:
df4 = df3.withColumn("PRODUCT_ID", regexp_replace("PRODUCT_ID", symbol_pattern, ""))

In [19]:
df4.show(n=20, truncate=False)

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|CP    |DISCOUNT|SP    |Date      |
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|YR7220  |New York      |Electronics     |12254943  |$31|$20.77|$1.86   |$29.14|2019-11-26|
|YR7220  |New York      |Furniture       |72619323C |$15|$9.75 |$1.5    |$13.5 |2019-11-26|
|YR7220  |New York      |Electronics     |34161682B |$88|$62.48|$4.4    |$83.6 |2019-11-26|
|YR7220  |New York      |Kitchen         |79411621  |$91|$58.24|$3.64   |$87.36|2019-11-26|
|YR7220  |New York      |Fashion         |39520263T |$85|$51   |$2.55   |$82.45|2019-11-26|
|YR7220  |New York      |Kitchen         |93809204  |$37|$24.05|$0.74   |$36.26|2019-11-26|
|YR7220  |New York      |Cosmetics       |86610412D |$80|$48.8 |$6.4    |$73.6 |2019-11-26|
|YR7220  |New York      |Kitchen         |52503356  |$71|$42.6 |$5.68   |$65.32|

In [20]:
df_pd = df4.limit(5).toPandas()

In [21]:
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   STORE_ID          5 non-null      object
 1   STORE_LOCATION    5 non-null      object
 2   PRODUCT_CATEGORY  5 non-null      object
 3   PRODUCT_ID        5 non-null      object
 4   MRP               5 non-null      object
 5   CP                5 non-null      object
 6   DISCOUNT          5 non-null      object
 7   SP                5 non-null      object
 8   Date              5 non-null      object
dtypes: object(9)
memory usage: 488.0+ bytes


In [22]:
df4.limit(5).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,New York,Electronics,12254943,$31,$20.77,$1.86,$29.14,2019-11-26
1,YR7220,New York,Furniture,72619323C,$15,$9.75,$1.5,$13.5,2019-11-26
2,YR7220,New York,Electronics,34161682B,$88,$62.48,$4.4,$83.6,2019-11-26
3,YR7220,New York,Kitchen,79411621,$91,$58.24,$3.64,$87.36,2019-11-26
4,YR7220,New York,Fashion,39520263T,$85,$51,$2.55,$82.45,2019-11-26


In [16]:
df.count()

515738

In [23]:
df4.select("Date").show(n=5, truncate=False)

+----------+
|Date      |
+----------+
|2019-11-26|
|2019-11-26|
|2019-11-26|
|2019-11-26|
|2019-11-26|
+----------+
only showing top 5 rows



In [58]:
df2.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: string (nullable = true)
 |-- CP: string (nullable = true)
 |-- DISCOUNT: string (nullable = true)
 |-- SP: string (nullable = true)
 |-- Date: date (nullable = true)



In [25]:
df4.limit(1).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,New York,Electronics,12254943,$31,$20.77,$1.86,$29.14,2019-11-26


In [61]:
df3 = df2.withColumn("Date", F.to_date(F.col("Date"),"M/d/yyyy"))

In [62]:
df3.limit(5).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,New York(,Electronics,12254943,$31,$20.77,$1.86,$29.14,2019-11-26
1,YR7220,New York+,Furniture,72619323C,$15,$9.75,$1.5,$13.5,2019-11-26
2,YR7220,New York,Electronics,34161682B,$88,$62.48,$4.4,$83.6,2019-11-26
3,YR7220,New York!,Kitchen,79411621,$91,$58.24,$3.64,$87.36,2019-11-26
4,YR7220,New York,Fashion,39520263T,$85,$51,$2.55,$82.45,2019-11-26


In [63]:
df3.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: string (nullable = true)
 |-- CP: string (nullable = true)
 |-- DISCOUNT: string (nullable = true)
 |-- SP: string (nullable = true)
 |-- Date: date (nullable = true)



# Define a Schema Manually: Method-2 (sql way)

In [45]:
sql_way_schema = " Hotel_Address string, \
Additional_Number_of_Scoring integer, \
Review_Date string, \
Average_Score float, \
Hotel_Name string, \
Reviewer_Nationality string, \
Negative_Review string, \
Review_Total_Negative_Word_Counts integer, \
Total_Number_of_Reviews string, \
Positive_Review string, \
Review_Total_Positive_Word_Counts integer, \
Total_Number_of_Reviews_Reviewer_Has_Given integer, \
Reviewer_Score float, \
Tags string, \
days_since_review string, \
lat float, \
lng float"

In [ ]:
spark.stop()